In [1]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

from pydataset import data
import pandas as pd
import sklearn.preprocessing
import wrangle as w


from sklearn.metrics import mean_squared_error

1. Load the tips dataset.

* Create a column named price_per_person. This should be the total bill divided by the party size.
* Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?
* Use select k best to select the top 2 features for predicting tip amount. What are they?
* Use recursive feature elimination to select the top 2 features for tip amount. What are they?
* Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features you are selecting?

In [10]:
tips = data("tips")

In [12]:
tips.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [13]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 1 to 244
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 15.2+ KB


In [46]:
tips["price_per_person"] = (tips.total_bill + tips.tip) / tips.party

In [50]:
# Rename size columns to party
# tips["party"] = tips["size"] 
# tips = tips.drop(columns=["size"])

In [59]:
# 2 cats
tips.sex.value_counts()
# 2 cats
tips.smoker.value_counts()
# 4 cats
tips.day.value_counts()
# 2 cats
tips.time.value_counts()
# 6 cats
tips.party.value_counts()

2    156
3     38
4     37
5      5
1      4
6      4
Name: party, dtype: int64

In [64]:
dummy_df = pd.get_dummies(tips[["sex", "smoker", "day", "time"]], drop_first=True)

In [67]:
tips = pd.concat([tips, dummy_df], axis=1)

In [134]:
# creat df for modeling and scaling 
# split/scaling 
# split data
train, validate, test = w.split_data(tips)


In [140]:
new_train = train.reset_index()

In [141]:
# columns to scale
new_train[["total_bill", "tip", "party", "price_per_person"]] 

,total_bill,tip,party,price_per_person
0,13.42,1.58,2,7.500000
1,23.33,5.65,2,14.490000
2,44.30,2.50,3,15.600000
3,17.51,3.00,2,10.255000
4,25.28,5.00,2,15.140000
5,16.82,4.00,2,10.410000
6,23.68,3.31,2,13.495000
7,7.51,2.00,2,4.755000
8,34.83,5.17,4,10.000000
9,16.04,2.24,3,6.093333


In [142]:
scale_df = new_train[["total_bill", "party", "price_per_person"]]
scaler = sklearn.preprocessing.MinMaxScaler()
scaler.fit(scale_df)


x_train_scaled = scaler.transform(scale_df)
# x_validate_scaled = scaler.transform(validate)
# x_test_scaled = scaler.transform(test)



In [143]:
col_name = ['total_bill', 'party', 'price_per_person']

In [144]:
x_train_scaled = pd.DataFrame(x_train_scaled)

In [145]:
x_train_scaled = x_train_scaled.rename(columns=dict(zip(x_train_scaled.columns, col_name)))


In [146]:
def rename_col(df, list_of_columns=[]): 
    '''
    Take df with incorrect names and will return a renamed df using the 'list_of_columns' which will contain a list of appropriate names for the columns  
    '''
    df = df.rename(columns=dict(zip(df.columns, list_of_columns)))
    return df

In [147]:
x_train_scaled = rename_col(x_train_scaled, col_name)

In [116]:
train.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'price_per_person',
       'party', 'sex_Male', 'smoker_Yes', 'day_Sat', 'day_Sun', 'day_Thur',
       'time_Lunch'],
      dtype='object')

In [157]:
encode_df = new_train[['sex_Male', 'smoker_Yes', 'day_Sat', 'day_Sun', 'day_Thur', 'time_Lunch']]

In [164]:
x_train = pd.concat([ x_train_scaled, encode_df], axis=1)

In [162]:
y_train = new_train[["tip"]]

In [216]:
col_name = ["total_bill", "party", "price_per_person", "male", "smoker", "day_sat", "day_sun", "day_thur", "lunch"]

In [217]:
x_train = rename_col(x_train, col_name)

In [218]:
eval_df = pd.concat([x_train,y_train], axis=1)

In [212]:
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings("ignore")


In [199]:
# creat baseline
def get_baseline(df, y):
    df["yhat_baseline"] = df[y].mean()
    return df

In [176]:


# generate parameters, i.e. create model
ols_model = LinearRegression().fit(x_train, y_train)

# compute predictions and add to original dataframe
eval_df['yhat'] = ols_model.predict(x_train)

In [177]:
eval_df

,tota_bill,party,price_per_person,male,smoker,day_sat,day_sun,day_thur,lunch,tip,yhat_baseline,yhat
0,0.228679,0.2,0.196906,1,1,0,0,0,1,1.58,2.981096,2.047353
1,0.447636,0.2,0.590155,1,1,0,1,0,0,5.65,2.981096,3.857762
2,0.910959,0.4,0.652602,0,1,1,0,0,0,2.50,2.981096,4.503985
3,0.319046,0.2,0.351899,0,1,0,1,0,0,3.00,2.981096,2.939907
4,0.490720,0.2,0.626723,0,1,1,0,0,0,5.00,2.981096,4.104203
5,0.303800,0.2,0.360619,1,1,0,1,0,0,4.00,2.981096,2.803385
6,0.455369,0.2,0.534177,1,0,0,1,0,0,3.31,2.981096,3.703980
7,0.098100,0.2,0.042475,1,0,0,0,1,1,2.00,2.981096,1.312334
8,0.701723,0.6,0.337553,0,0,0,0,1,1,5.17,2.981096,4.471479
9,0.286567,0.4,0.117768,1,0,1,0,0,0,2.24,2.981096,2.782899


In [185]:
def get_sse(df, x, y)
    df = get_baseline(df, y)
    SSE = mean_squared_error(df.tip, df.yhat)*len(eval_df)
    SSE_baseline = mean_squared_error(eval_df.tip, eval_df.yhat_baseline)*len(eval_df)

MSE = mean_squared_error(eval_df.tip, eval_df.yhat)

MSE_baseline = mean_squared_error(eval_df.tip, eval_df.yhat_baseline)

In [186]:

RMSE = sqrt(mean_squared_error(eval_df.tip, eval_df.yhat))
RMSE_baseline = sqrt(mean_squared_error(eval_df.tip, eval_df.yhat_baseline))

In [188]:

df_eval = pd.DataFrame(np.array(['SSE','MSE','RMSE']), columns=['metric'])
df_baseline_eval = pd.DataFrame(np.array(['SSE_baseline','MSE_baseline','RMSE_baseline']), columns=['metric'])

df_eval['model_error'] = np.array([SSE, MSE, RMSE])
df_baseline_eval['model_error'] = np.array([SSE_baseline, MSE_baseline, RMSE_baseline])

In [190]:
df_eval['error_delta'] = df_eval.model_error - df_baseline_eval.model_error
df_eval

,metric,model_error,error_delta
0,SSE,144.163686,-163.269538
1,MSE,0.987423,-1.118285
2,RMSE,0.993691,-0.457414


In [191]:
df_baseline_eval

,metric,model_error
0,SSE_baseline,307.433225
1,MSE_baseline,2.105707
2,RMSE_baseline,1.451105


In [195]:


evs = explained_variance_score(eval_df.tip, eval_df.yhat)
print('Explained Variance = ', round(evs,3))

Explained Variance =  0.531


In [221]:
from sklearn.feature_selection import SelectKBest, f_regression

# parameters: f_regression stats test, give me 2 features
f_selector = SelectKBest(f_regression, k=2)

# find the top 8 X's correlated with y
f_selector.fit(x_train, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = x_train.iloc[:,feature_mask].columns.tolist()


In [222]:
f_feature

['total_bill', 'party']

In [223]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, n_features_to_select=2)

# fit the data using RFE
rfe.fit(x_train,y_train)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = x_train.iloc[:,feature_mask].columns.tolist()


In [224]:
rfe_feature

['party', 'price_per_person']

In [235]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_regression, RFE

def select_kbest(x, y, k):
    selector = SelectKBest(f_regression, k=k)
    selector.fit(x, y)
    selected_features_mask = selector.get_support()
    selected_features = x.columns[selected_features_mask].tolist()
    return selected_features


In [237]:
import pandas as pd

from sklearn.linear_model import LinearRegression

def rfe(x, y, k):
    estimator = LinearRegression()  # Estimator used for feature ranking
    selector = RFE(estimator, n_features_to_select=k)
    selector.fit(x, y)
    selected_features_mask = selector.support_
    selected_features = x.columns[selected_features_mask].tolist()
    return selected_features



In [238]:
rfe(x_train, y_train, 2)

['party', 'price_per_person']

In [236]:
select_kbest(x_train, y_train, 2)

['total_bill', 'party']

In [2]:
import feature_engineering as fe
from pydataset import data

In [3]:
swiss = data("swiss")

In [4]:
swiss.columns = swiss.columns.str.lower()

In [5]:
swiss.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, Courtelary to Rive Gauche
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fertility         47 non-null     float64
 1   agriculture       47 non-null     float64
 2   examination       47 non-null     int64  
 3   education         47 non-null     int64  
 4   catholic          47 non-null     float64
 5   infant.mortality  47 non-null     float64
dtypes: float64(4), int64(2)
memory usage: 2.6+ KB


In [31]:
# 2 cats  - bool
# swiss.fertility.value_counts()

# #  0 cats  
# swiss.agriculture.value_counts()

# # 0 cats  
# swiss.examination.value_counts()

# # 0 cats  
# swiss.education.value_counts()

# # # 2 cats  - bool
# swiss.catholic.value_counts()

# # 3 cats
# swiss["infant.mortality"].value_counts()

In [53]:
# min max data and split into the 3 training sets 
x_train_scaled, x_validate_scaled, x_test_scaled = w.mm_scale(swiss[['agriculture', 'examination', 'education', 'catholic', 'infant.mortality']])

In [55]:
col_name = [ 'agriculture', 'examination', 'education', 'catholic', 'infant.mortality']


In [60]:
y_train, y_validate, y_test = w.split_data(swiss["fertility"])

In [56]:
# fic column names after splitting an d scaling
x_train_scaled, x_validate_scaled, x_test_scaled = pd.DataFrame(x_train_scaled), pd.DataFrame(x_validate_scaled), pd.DataFrame(x_test_scaled)

x_train_scaled, x_validate_scaled, x_test_scaled  = w.rename_col(x_train_scaled, col_name), w.rename_col(x_validate_scaled, col_name), w.rename_col(x_test_scaled, col_name)

In [57]:
x_train_scaled

,agriculture,examination,education,catholic,infant.mortality
0,0.474390,0.40625,0.903226,0.492786,0.316327
1,0.729268,0.34375,0.193548,0.000000,0.408163
2,0.647561,0.40625,0.290323,0.055766,0.122449
3,0.796341,0.50000,0.258065,0.005832,0.500000
4,0.526829,0.59375,0.354839,0.131689,0.163265
5,0.953659,0.00000,0.032258,0.997033,0.000000
6,0.509756,0.37500,0.225806,0.039190,0.755102
7,0.774390,0.28125,0.000000,0.001330,0.602041
8,0.751220,0.59375,0.129032,0.005628,0.367347
9,0.831707,0.18750,0.258065,0.990382,0.275510


In [61]:
y_train

Rive Droite     44.7
Aubonne         66.9
Rolle           60.5
Lavaux          65.1
Nyone           56.6
Conthey         75.5
Yverdon         65.4
Oron            72.5
Cossonay        61.7
St Maurice      65.0
Franches-Mnt    92.5
Orbe            57.4
Sarine          82.9
La Chauxdfnd    65.7
Neuchatel       64.4
Monthey         79.4
Broye           83.8
Paysd'enhaut    72.0
Le Locle        72.7
Sion            79.3
Herens          77.3
Glane           92.4
Neuveville      76.9
Grandson        71.7
Vevey           58.3
Martigwy        70.5
Avenches        68.9
Name: fertility, dtype: float64

In [63]:
fe.select_kbest(x_train_scaled, y_train, 3)

['examination', 'catholic', 'infant.mortality']

In [64]:
fe.rfe(x_train_scaled, y_train, 3)

['examination', 'education', 'infant.mortality']